In [504]:
###  Script to apply feature factor weights identified in Munich Data (Azimuth Annotation) on Groningen data for computation of factor values
###  Replication Script

# Libraries

In [505]:
library(dplyr)
library(stringr)
library(Seurat)

In [506]:
library(MOFA2)

In [507]:
library(ggplot2)
library(reticulate)
library(reshape2)
library(ggpubr)

# Parameters

In [508]:
data_path = '../data/current'

In [509]:
output_path = '../results/current'

# Load data

## Munich factor data

In [510]:
factor_data_mu = read.csv(paste0(data_path, "/results/G-Analysis/Factor_Data_V_AZIMUTH_INTEGRATED_FALSE.csv"))
head(factor_data_mu)  # alternative: Azimuth

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.2822028,-1.2559541,-0.53390026,1.1204467,-0.46669223,1.2484388,0.20759032,-0.364617945,-0.40242258,0.78832135,⋯,0.05325919,0.27738239,0.13303365,-0.76325230,-0.04660508,0.95971046,0.431779125,-0.46741569,-0.45888943,k1
2,0.6928825,-1.0380557,0.33933545,0.2294335,0.10520940,0.8011520,0.36775010,1.417721825,-0.58040927,0.04100113,⋯,0.22233569,0.31122340,0.04224790,0.14198857,-0.04503017,-0.04460783,-0.186394597,-0.26701984,-0.03163122,k10
3,-1.4468008,-0.3123073,0.68857816,-0.6941110,0.08007671,1.1839352,-0.19331847,0.036830866,0.21506213,0.03104541,⋯,-0.70126043,0.17834780,-0.05727359,0.05895828,0.14746604,-0.09273922,0.172580802,0.02947317,-0.08184409,k11
4,-2.5657673,-0.7162750,0.99399349,-1.0877413,-0.89743312,0.5699462,-0.08501748,-0.168697397,0.03162881,-0.03488845,⋯,-1.09175933,0.48647681,-0.39887633,-0.04494189,0.54622186,0.16467157,-0.149100299,0.43129440,-0.05209181,k12
5,-0.2657781,-0.1293536,0.30498721,-1.0304524,0.04534561,1.3639342,0.02831011,-0.003340543,0.13941860,0.16450688,⋯,-0.33919680,-0.66542364,0.03761243,0.15980442,0.11184416,0.03747477,0.099951109,0.24887446,0.27424704,k13
6,1.0376091,-1.4776762,-0.03257904,-0.5101523,0.12084320,0.1672229,0.24861033,1.302995675,0.07186483,0.02188782,⋯,0.56497282,-0.05101193,-0.29314109,0.26959618,-0.70380578,0.07279563,0.004502305,0.38742801,0.05504769,k14


## Munich feature data

In [518]:
feature_data_mu = read.csv(paste0(data_path, "/results/G-Analysis/Weight_Data_V_AZIMUTH_INTEGRATED_FALSE.csv"))
head(feature_data_mu)  # alternative: Munich

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,type,variable_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,-0.020691979,0.429515363,0.027756312,-0.02712267,0.010438305,-0.41947963,0.01169812,-0.11248843,0.28675204,0.0143456634,⋯,0.030966401,0.100109005,0.88420419,-0.002553659,-0.0002680875,-0.009308180,0.003408993,-0.002008598,clinical_data,CK
2,-0.001361949,0.379039595,0.067498675,-0.06765357,-0.004929694,-0.35290690,0.01561328,-0.01496201,0.37774439,-0.0004131968,⋯,0.003047890,0.116174613,0.13072112,-0.002865083,-0.0003610348,-0.008757576,0.004192790,-0.010507527,clinical_data,CK_MB
3,-0.052856754,0.028641115,-0.891960581,0.08775674,-0.024844550,-0.40121965,-0.08041434,-0.01238686,-0.02412622,-0.0140435079,⋯,0.008406062,0.176649898,0.49146629,-0.007964040,-0.0036451100,0.014286835,-0.002485743,0.009543655,clinical_data,CRP
4,-0.026301185,0.327099511,-0.314584680,0.08342660,0.014994718,-0.54065063,0.01323074,-0.31178849,0.18761051,-0.0154090508,⋯,0.117386061,0.216599600,0.78670880,-0.007199139,0.0443780006,-0.003896878,0.004977378,-0.003467445,clinical_data,Troponin
5,0.015548781,-0.004530304,0.082117144,-0.02945236,-0.213756108,-0.10763285,0.07139866,0.03609601,-0.03866970,0.0178796863,⋯,0.262153876,0.006848431,-0.05089458,-0.020458287,-0.1043864240,0.075482525,-0.037997785,-0.029645511,cytokine,BCA1__CXCL13
6,0.015036488,-0.190978857,0.001049874,0.01339136,0.028743482,0.01589764,0.07016133,-0.02435615,0.01165956,-0.1114046521,⋯,-0.001984003,-0.050574208,-0.04242095,-0.004868774,-0.1907216033,0.364199683,0.085955948,-0.028747833,cytokine,CTACK__CCL27


## Pseudobulk and Normalized Input Data Groningen

In [519]:
path = paste0(output_path, '/Reproduction_GR/Combined_Data_V_AZIMUTH_REPRODUCTION_v2','.csv')
input_gr = read.csv(path)
head(input_gr, n=3)
print(file.info(path)$mtime)

,X,sample_id,variable,value,type
,<int>,<chr>,<chr>,<dbl>,<chr>
1,1,Baseline.TEST_1,B.cell__ACTB,-0.1451209,single_cell
2,2,Baseline.TEST_12,B.cell__ACTB,0.2310092,single_cell
3,3,Baseline.TEST_14,B.cell__ACTB,-0.2933812,single_cell


[1] "2023-04-25 16:06:43 UTC"


In [520]:
input_gr$feature = input_gr$variable

## Outcome classification Groningen

In [ ]:
classification_gr = read.csv(paste0(data_path, "/results/Reproduction_GR/groningen_output_class.csv"))

# Analyze

## Match Features

In [522]:
### Ensure that in both datasets feature annotations including cell-type have same format

In [523]:
input_gr$cell_type = str_replace(input_gr$variable,
                                 '__.*', '')

In [524]:
unique(input_gr$cell_type)

[1] "B.cell"    "CD4_CTL"   "CD4_Naive" "CD4_TCM"   "CD4_TEM"   "CD8_Naive"
 [7] "CD8_TEM"   "cDC2"      "cMono"     "MAIT"      "ncMono"    "NKdim"    
[13] "Treg"

In [525]:
## remove non-sc features from Munich data

In [526]:
head(feature_data_mu,2)
unique(feature_data_mu$type)
feature_data_mu = feature_data_mu[
  !feature_data_mu$type %in% c('clinical_data', 'cytokine', 'neutrophil', 'proteomics'),
]

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,type,variable_name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,-0.020691979,0.4295154,0.02775631,-0.02712267,0.010438305,-0.4194796,0.01169812,-0.11248843,0.2867520,0.0143456634,⋯,0.03096640,0.1001090,0.8842042,-0.002553659,-0.0002680875,-0.009308180,0.003408993,-0.002008598,clinical_data,CK
2,-0.001361949,0.3790396,0.06749868,-0.06765357,-0.004929694,-0.3529069,0.01561328,-0.01496201,0.3777444,-0.0004131968,⋯,0.00304789,0.1161746,0.1307211,-0.002865083,-0.0003610348,-0.008757576,0.004192790,-0.010507527,clinical_data,CK_MB


[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
 [5] "Bcell"         "CD14Mono"      "CD16Mono"      "CD4CTL"       
 [9] "CD4Naive"      "CD4TCM"        "CD4TEM"        "CD8Naive"     
[13] "CD8TEM"        "cDC2"          "gdT"           "NK"           
[17] "Treg"

In [527]:
length(unique(feature_data_mu$variable_name))

[1] 7694

In [528]:
## Adjust feature names to align with Groningen Mapping

In [529]:
#### Azimuth
feature_data_mu$cell_type = str_replace(feature_data_mu$variable_name,
                                        '__.*', '')

unique(feature_data_mu$cell_type)

feature_data_mu$feature = feature_data_mu$variable_name

feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "CD14.Mono", 'cMono')


feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "Bcell", 'B.cell')

feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "CD16.Mono", 'ncMono')

feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "CD4.TCM", 'CD4_TCM')

feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "cDC2", 'cDC2')

feature_data_mu$feature = str_replace(
  feature_data_mu$feature, "NK", 'NKdim')




head(feature_data_mu,2)
length(unique(feature_data_mu$feature))

[1] "B.cell"    "CD14.Mono" "CD16.Mono" "CD4.CTL"   "CD4.Naive" "CD4.TCM"  
 [7] "CD4.TEM"   "CD8.Naive" "CD8.TEM"   "cDC2"      "gdT"       "NK"       
[13] "Treg"

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,type,variable_name,cell_type,feature
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1452,-0.17926582,0.23935213,0.1632959,-0.04758595,0.13904403,-0.06905755,1.1085302,0.012131219,0.3152998,0.04606748,⋯,-0.01115103,0.054059598,-0.2188138,-0.1313958,-0.03007437,0.1566817,Bcell,B.cell__ACTB,B.cell,B.cell__ACTB
1453,-0.04518202,-0.01996126,0.1115015,-0.01237420,0.07304104,-0.07186289,0.6682053,0.002994229,0.1844658,-0.03832849,⋯,-0.02316688,-0.006085256,-0.5234638,-0.3893344,-0.17256498,-0.4521462,Bcell,B.cell__ACTG1,B.cell,B.cell__ACTG1


[1] 7694

In [530]:
### Compare

In [531]:
#sort(unique(str_replace(feature_data_mu$feature, '__.*', ''))) # Munich
#sort(unique(input_gr$cell_type)) # Groningen

In [532]:
### Long format of mu feature data

In [533]:
feature_data_mu = melt(feature_data_mu)

Using type, variable_name, cell_type, feature as id variables



In [534]:
head(feature_data_mu,2)

,type,variable_name,cell_type,feature,variable,value
,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,Bcell,B.cell__ACTB,B.cell,B.cell__ACTB,Factor1,-0.17926582
2,Bcell,B.cell__ACTG1,B.cell,B.cell__ACTG1,Factor1,-0.04518202


In [535]:
# Filter MU feature weights for features also in GR data

In [536]:

feature_data_mu = feature_data_mu[feature_data_mu$feature %in% unique(input_gr$feature),]
head(feature_data_mu,2)
length(unique(feature_data_mu$feature))

,type,variable_name,cell_type,feature,variable,value
,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>
1,Bcell,B.cell__ACTB,B.cell,B.cell__ACTB,Factor1,-0.1792658
24,Bcell,B.cell__ARHGDIB,B.cell,B.cell__ARHGDIB,Factor1,0.3273940


[1] 3230

In [537]:
sort(unique(str_replace(feature_data_mu$feature, '__.*', '')))   # check mapped cell-types

[1] "B.cell"  "CD4_TCM" "cDC2"    "cMono"   "ncMono"  "NKdim"   "Treg"

In [538]:
feature_data_mu = feature_data_mu[feature_data_mu$cell_type != 'Treg',] # exclude not so well mapped cell-type

In [539]:
length(unique(feature_data_mu$feature))

[1] 3148

In [540]:
# Filter GR input features for features also in MU data

In [541]:
#head(input_gr)
length(unique(input_gr$feature))
input_gr = input_gr[input_gr$feature %in% unique(feature_data_mu$feature), ]
input_vis = input_gr
input_gr = input_gr %>% dcast(sample_id ~ feature, value.var =  'value')

[1] 6353

In [542]:
head(input_gr)
dim(input_gr)

,sample_id,B.cell__ACTB,B.cell__ARHGDIB,B.cell__ARPC2,B.cell__ARPC3,B.cell__B2M,B.cell__BIRC3,B.cell__BTF3,B.cell__BTG1,B.cell__BTG2,⋯,NKdim__YPEL5,NKdim__YWHAB,NKdim__YWHAQ,NKdim__YWHAZ,NKdim__ZC3HAV1,NKdim__ZEB2,NKdim__ZFAS1,NKdim__ZFP36,NKdim__ZFP36L1,NKdim__ZFP36L2
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Baseline.TEST_1,-0.1451209,-0.2808230,0.2933812,-1.3037827,0.35691713,-0.29338123,-0.5720211,1.6637928,-1.3037827,⋯,1.6637928,0.70506589,0.4752408,0.9234567,0.6744898,-0.02410453,-1.0405662,-0.94207577,-0.80109453,-0.7050659
2,Baseline.TEST_12,0.2310092,-0.0602920,0.3697907,-0.3697907,-0.29338123,-0.08445798,0.4752408,0.1451209,0.1573107,⋯,-0.6744898,-0.09655862,0.8178173,-0.3827258,-0.7363159,-0.44842548,-1.8980287,-0.07236971,1.36238339,0.8178173
3,Baseline.TEST_14,-0.2933812,0.1329527,1.6637928,-0.6151411,0.96102726,1.53412054,1.3935235,1.0405662,-1.0829150,⋯,1.7688250,0.34410246,0.7363159,-0.1817624,-1.5341205,-0.61514110,-0.1451209,-1.06150263,-1.76882504,0.5299395
4,Baseline.TEST_15,-1.0615026,-0.4087920,-0.7206029,-0.2683089,-0.08445798,-0.38272581,-1.2493462,-1.0405662,1.1740260,⋯,0.6744898,-0.58627265,0.6445316,0.6297684,-0.3313441,0.60064430,0.2683089,-1.15034938,-0.04822307,-0.8694238
5,Baseline.TEST_17,-1.1503494,-1.4602158,-0.5299395,-0.9420758,0.78459288,0.52993955,1.5744450,-0.4219289,1.0829150,⋯,0.7363159,-1.19837970,-0.1817624,-1.1503494,0.1086734,-0.39572530,-1.3037827,0.20632319,-1.66379279,1.4602158
6,Baseline.TEST_18,1.1273007,0.6297684,2.3410271,0.2063232,1.61750540,0.68969722,-1.5341205,0.4617916,-1.5744450,⋯,-0.3827258,-0.73631592,-0.3186394,1.3037827,0.9234567,1.49614688,-1.2493462,-0.73631592,-1.30378267,1.4602158


[1]  103 3149

## Prepare matrices

In [543]:
### Get matrices for matrix facorization approach

## Groningen Matrix
#head(input_gr)
gr_ma = input_gr
rownames(gr_ma) = input_gr$sample_id
gr_ma$sample_id = NULL
gr_ma = as.matrix(gr_ma)
dim(gr_ma)

## Munich Matrix
#head(feature_data_mu)
mu_ma = feature_data_mu %>% dcast(feature ~variable, value.var =  'value')
rownames(mu_ma) = mu_ma$feature
mu_ma$feature = NULL
mu_ma = as.matrix(mu_ma)


[1]  103 3148

In [544]:
dim(gr_ma)
dim(mu_ma)

[1]  103 3148

[1] 3148   20

In [545]:
### Reorder to have same ordering of features

In [546]:
gr_ma = gr_ma[, order(colnames(gr_ma))]
mu_ma = mu_ma[order(rownames(mu_ma)),]

## Calculate right inverse of MU feature matrix and apply

In [547]:
### Calculate right inverse of weight matrix (mu_ma)

dim(mu_ma)  ## corresponds to A^T
mu_ma_t = t(mu_ma)  
dim(mu_ma_t) ## corresponds to # A

mu_mat_right_inv = mu_ma %*% (solve(mu_ma_t %*% mu_ma))
head(mu_mat_right_inv)
tail(mu_mat_right_inv)
dim(mu_mat_right_inv)


[1] 3148   20

[1]   20 3148

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20
B.cell__ACTB,-0.002412865,1.100829e-03,0.0018519132,-9.387865e-05,0.0197072832,-1.171368e-03,0.004082177,3.549377e-04,0.0012770596,-0.0004148242,0.017505496,8.850074e-04,-5.435629e-04,9.115598e-04,-1.148509e-05,2.181989e-04,-1.067792e-03,-0.0003216798,-0.0005795758,0.0007411074
B.cell__ARHGDIB,0.002896903,-3.862585e-04,0.0011406868,1.345452e-03,0.0149805072,1.937177e-04,0.001627769,-2.946817e-04,0.0001059237,0.0006040866,-0.014150740,2.117682e-04,-2.951694e-04,-5.412122e-04,-1.305719e-04,-5.945591e-05,-2.291039e-03,0.0001882649,-0.0002861554,0.0002756893
B.cell__ARPC2,-0.001114492,-6.099780e-05,0.0003215208,4.180816e-04,0.0209911475,-3.725421e-04,0.003521452,2.003061e-04,0.0013498674,-0.0020104846,0.018957908,-6.100954e-04,-6.614388e-05,2.495120e-04,-2.905480e-04,1.102433e-03,-2.069296e-03,-0.0005396076,0.0004881568,-0.0004772543
B.cell__ARPC3,0.001612616,-1.623649e-05,0.0008018460,3.144724e-04,0.0432317206,-2.770284e-03,0.002947859,-2.522687e-04,-0.0009838727,-0.0028467784,-0.022270218,2.108312e-05,7.876929e-04,2.812663e-03,-3.912311e-04,2.077665e-03,7.316946e-05,0.0002948308,0.0005419943,-0.0002239274
B.cell__B2M,-0.001969751,9.628387e-04,0.0012608342,5.635957e-05,-0.0039711670,-4.673417e-04,0.003398598,5.528947e-05,0.0030406779,-0.0009352855,0.006424656,1.930557e-03,-1.285816e-04,6.238179e-05,1.034847e-04,9.638252e-05,-2.213990e-03,-0.0004096560,0.0010927847,0.0014692351
B.cell__BIRC3,0.001618585,-5.811380e-05,0.0007165601,-1.982736e-03,0.0008224573,-9.228915e-05,-0.002526311,-1.446628e-04,-0.0032944132,0.0005858396,-0.042494057,1.831863e-03,3.520557e-04,-9.515494e-04,1.258441e-03,-4.166334e-04,-4.329759e-03,0.0009212626,0.0013008816,0.0012679802


,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20
NKdim__ZC3HAV1,-4.191599e-03,4.440209e-04,0.0002153844,0.0004540283,0.0002949058,-0.0007085776,0.0004619467,0.0009136763,0.0017003505,0.0002068845,0.029082471,-0.0013899221,0.0005879015,-0.0017232555,0.0002550067,0.0020538919,-0.0001903588,-0.001723843,-0.0001421164,-1.611028e-04
NKdim__ZEB2,-4.169701e-06,2.449899e-03,0.0024518547,-0.0006276671,-0.0006177782,0.0003662261,0.0002654389,-0.0001024299,-0.0011893979,0.0029621473,0.008816921,0.0004274647,0.0017044722,-0.0002884657,-0.0002698381,-0.0009618766,-0.0013183011,0.001507303,0.0001103782,2.288160e-04
NKdim__ZFAS1,-1.824321e-03,-2.196386e-05,-0.0001309291,0.0002609197,0.0031960898,-0.0009808865,0.0003104403,-0.0003755947,-0.0001760204,-0.0030818508,-0.113659437,-0.0030505201,-0.0005130803,-0.0014203910,0.0004811148,0.0003157216,-0.0009303368,0.001418756,0.0007723607,2.154406e-04
NKdim__ZFP36,-1.884174e-03,-1.013808e-04,-0.0009070395,0.0004174070,0.0002005844,0.0019932858,0.0003703909,0.0008108770,-0.0016578828,-0.0039247834,0.008802220,0.0007937521,0.0021959706,-0.0009571731,-0.0004312455,0.0007124782,0.0005021595,-0.004344592,0.0007678170,-2.155689e-04
NKdim__ZFP36L1,-2.860396e-03,1.044763e-04,0.0009260348,-0.0028899986,0.0022447461,0.0009851776,0.0015616902,0.0010531281,-0.0021344237,-0.0053043853,-0.004016054,0.0004029870,0.0009022350,-0.0020380839,0.0004893038,0.0004537000,0.0002285394,-0.000127352,-0.0014641171,1.223379e-05
NKdim__ZFP36L2,-1.056855e-03,-6.163430e-04,0.0018555297,0.0006076941,0.0017680046,0.0007874596,0.0007943166,0.0003794940,0.0004912557,-0.0051923213,0.080864937,-0.0008642006,0.0014579452,-0.0028402238,0.0004304173,0.0003052706,-0.0003380432,-0.002692252,-0.0017124035,-6.965453e-05


[1] 3148   20

In [548]:
head(gr_ma,2)

,B.cell__ACTB,B.cell__ARHGDIB,B.cell__ARPC2,B.cell__ARPC3,B.cell__B2M,B.cell__BIRC3,B.cell__BTF3,B.cell__BTG1,B.cell__BTG2,B.cell__CALM1,⋯,NKdim__YPEL5,NKdim__YWHAB,NKdim__YWHAQ,NKdim__YWHAZ,NKdim__ZC3HAV1,NKdim__ZEB2,NKdim__ZFAS1,NKdim__ZFP36,NKdim__ZFP36L1,NKdim__ZFP36L2
Baseline.TEST_1,-0.1451209,-0.280823,0.2933812,-1.3037827,0.3569171,-0.29338123,-0.5720211,1.6637928,-1.3037827,-1.1048357,⋯,1.6637928,0.70506589,0.4752408,0.9234567,0.6744898,-0.02410453,-1.040566,-0.94207577,-0.8010945,-0.7050659
Baseline.TEST_12,0.2310092,-0.060292,0.3697907,-0.3697907,-0.2933812,-0.08445798,0.4752408,0.1451209,0.1573107,0.3827258,⋯,-0.6744898,-0.09655862,0.8178173,-0.3827258,-0.7363159,-0.44842548,-1.898029,-0.07236971,1.3623834,0.8178173


In [549]:
gr_ma[is.na(gr_ma)]= 0

In [550]:
#### Apply right inverse to Y from GR 

result = gr_ma %*% mu_mat_right_inv
dim(result)


head(result,2)

[1] 103  20

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20
Baseline.TEST_1,-0.12063079,1.4305362,0.1444684,0.5318507,-0.1235437,0.12083592,0.2016771,0.2368509,-0.21416826,0.21182132,0.3961745,-0.3276672,0.4171498,0.3391938,-0.01248597,0.08640766,-0.02221648,-0.4124799,0.195775378,-0.1918527
Baseline.TEST_12,-0.09882374,0.4013937,0.3206466,0.5860041,-0.2155669,0.08823914,-0.1322515,0.2131773,0.04916782,0.07366254,3.4977509,-0.2186651,0.4484445,0.2666195,-0.13561254,0.53762915,0.02581057,-0.2500909,0.008599943,-0.1842714


In [551]:
nrow(result)

[1] 103

In [552]:
result = as.data.frame(result)

In [553]:
nrow(result[!is.na(result$Factor1),])

[1] 103

In [554]:
result = result[!is.na(result$Factor1),]

In [555]:
nrow(result)

[1] 103

In [561]:
### Save Factor Pattern to Share

write.csv(result, paste0(output_path, 'Factor_Data_Groningen.csv'))
